## Problem-9
Find the Pythagorean triplet (a, b, c) such that $a^2 + b^2 = c^2$ and $a + b + c = 1000$. Report the product $abc$.

In [1]:
import time
import math

## Brute Force

This feels like a simple problem where the triplet $(a, b, c)$ should satisfy two conditions:
* $a^2 + b^2 = c^2$, and
* $a + b + c = 1000$

In [2]:
def is_pythagorean_triplet(a, b, c):
    return a**2 + b**2 == c**2

# test method
print(is_pythagorean_triplet(3,4,5))
print(is_pythagorean_triplet(4,5,6))

True
False


In [3]:
def compute_pythgorean_triplet_for_given_sum(sum, print_results=False):
    condition_count = 0
    for a in range(1, sum):
        for b in range(1, sum):
            condition_count = condition_count + 1
            if is_pythagorean_triplet(a, b, 1000-a-b):
                if print_results:
                    result = (a, b, 1000-a-b)
                    print("Triplet: %s" % (result,))
                    print("Count: %d" % condition_count)
                return a * b * (1000-a-b)
            
start = time.time()
num = 1000
result = compute_pythgorean_triplet_for_given_sum(num, print_results=True)
end = time.time()
print("Required product of pythogorean triplet: %s" % result)
print("Time taken: %f sec" % (end - start))

Triplet: (200, 375, 425)
Count: 199176
Required product of pythogorean triplet: 31875000
Time taken: 0.201677 sec


## Discussion

For the specific case above ($sum = 1000$), maximum loop count would be $1000 \times 1000 = 1000000$, actual count is $31875000$ and as ratio, it is approximately $0.199$ or $0.2$ and the compute time is within 180 ms.

## Improvements:

### Improvement I:

* **Triangular Inequality**: $a + b > c$
* Combining with the condition $a + b + c = 1000$, we have: $c < 500$ and $a + b > 500$.
* Also, $c > a$ and $c > b$ which makes both of $a$ and $b$ individually less than $500$ too.
* We note that $a$ cannot be equal to $b$ since if $a = b$, it will imply that $c^2 = 2a^2$, which further implies that $c = a \sqrt{2}$, thus rendering c an irrational number.
* Assuming, without loss of generality, that $a < b$, we can limit search for $a$ on the lower side of $\frac{c}{\sqrt{2}}$ and for $b$ on the upper side of $\frac{c}{\sqrt{2}}$. The next closest integer for $\frac{c}{\sqrt{2}}$ is 354, assuming $c = 500$.
* If we check the following range for $b$: $[354, 500)$, the corresponding range for $a$ is: (1, 354].

In [4]:
def compute_pythgorean_triplet_for_given_sum_improved_1(sum, print_results=False):
    condition_count = 0
    max_c = int(sum/2)
    b_end = max_c
    b_start = int(max_c/math.sqrt(2))
    a_end = b_start
    a_start = 1
    for a in range(a_start, a_end):
        for b in range(b_start, b_end):
            condition_count = condition_count + 1
            if is_pythagorean_triplet(a, b, 1000-a-b):
                if print_results:
                    result = (a, b, 1000-a-b)
                    print("Triplet: %s" % (result,))
                    print("Count: %d" % condition_count)
                return a * b * (1000-a-b)
            
start = time.time()
num = 1000
result = compute_pythgorean_triplet_for_given_sum_improved_1(num, print_results=True)
end = time.time()
print("Required product of pythogorean triplet: %s" % result)
print("Time taken: %f sec" % (end - start))

Triplet: (200, 375, 425)
Count: 29276
Required product of pythogorean triplet: 31875000
Time taken: 0.029637 sec


## Discussion

Therefore, here the actual compute as a ratio is close to 0.029 which is approximately a tenth of the above method and the compute time is within 33 ms which shows an improvement by a factor of 6 approximately. But there can be further improvements.

### Improvement II:

* Leveraging Euclid's method of computing Pythagorean triplets [Wiki] (https://en.wikipedia.org/wiki/Pythagorean_triple#Generating_a_triple), we have: $a = k(m^2-n^2)$, $b = 2kmn$, $c = k(m^2 + n^2)$.

* Using the condition $a + b + c = sum$, we have: $2k(m^2 + mn) = 2km(m+n) = sum$, which implies there are 3 factors of $\frac{sum}{2}$: $k$, $m$, $m+n$.

* Given that $k$, $m$ and $m+n$ are factors of $\frac{sum}{2}$, we can say that $m(m+n) \le \frac{sum}{2}$ and thus $m^2 < \frac{sum}{2}$, and thus:
$$m < \sqrt{\frac{sum}{2}}$$

* For the given case here, we have $m < 22.36$ and so we need to check for values of $m$ in range $(1, 23)$.

* Similarly, working for $m+n$, from the Euclid's triplets, we had assumed that $m$ is greater than $n$, and thus we can say that $m < m+n < 2m$. Also, we have another bound on $m+n$ through the same equation as above: $km(m+n) = \frac{sum}{2}$, which further implies: $m+n < \frac{sum}{2m}$.

* Since k is the divisor and $m$ and $m+n$ are primes, GCD(m, m+n) = 1.

* Therefore, we can finally put all these together to obtain a solution.

In [5]:
def GCD(a, b):
    if b > a:
        return GCD(b, a)
    while a % b != 0:
        temp = a % b
        a = b
        b = temp % a
    return b

print(GCD(4,8))
print(GCD(35,49))

4
7


In [6]:
# let factor1 be m and thus factor1_max is sqrt(sum/2).
# let factor2 be m+n.
def compute_pythgorean_triplet_for_given_sum_improved_2(s, print_results=False):
    condition_count = 0
    factor1_max = math.ceil(math.sqrt(s/2))
    for factor1 in range(2, factor1_max):
        # if factor1 is a divisor for sum/2, then try to find factor2.
        if s/2 % factor1 == 0:
            # if factor1 is even, then factor 2 is odd.
            # else factor is odd.
            if factor1 % 2 == 0:
                factor2 = factor1 + 1
            else:
                factor2 = factor1 + 2
            while factor2 < 2*factor1 and factor2 <= s/(2*factor1):
                condition_count = condition_count+1
                if s/(2*factor1) % (factor2) == 0 and GCD(factor1, factor2) == 1:
                    k = s/(2*factor1*factor2)
                    m = factor1
                    n = factor2 - factor1
                    a = k * (m**2 - n**2)
                    b = 2 * k * m * n
                    c = k * (m**2 + n**2)
                    result = (int(a),int(b),int(c))
                    print ("Triplet: %s" % (result,))
                    print("Count: %d" % condition_count)
                    return a * b * (1000-a-b)
                factor2 = factor2 + 2
        else:
            condition_count = condition_count + 1
    
    
start = time.time()
num = 1000
result = compute_pythgorean_triplet_for_given_sum_improved_2(num, print_results=True)
end = time.time()
print("Required product of pythogorean triplet: %s" % result)
print("Time taken: %f sec" % (end - start))

Triplet: (375, 200, 425)
Count: 3
Required product of pythogorean triplet: 31875000.0
Time taken: 0.000198 sec


## Discussion

Therefore, here the actual compute as a ratio is close to 9.4e-8 which is way better than the previous improved method and the compute time is within 0.3 ms which shows an improvement by a factor of 100 approximately!!